# Описание обучения

### Данные

1. Берутся только id героев как фичи

### Обучение

1. Для **ML подходов:** \
Конкатинируются в вектор длины $ 2 * len(unique(id)) $ \
Для Radiant - $ len(unique(id)) $ \
Для Dire - $ len(unique(id)) $

2. Для **DL подходов:** ?

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/home/ankamenskiy/SmartDota/')

In [80]:
import pickle
import dataclasses
import json

import pandas as pd
import numpy as np
import torch

from src.data.api.download import ProMatchesDataloader

## Читаем данные

In [115]:
data_path = '/home/ankamenskiy/SmartDota/cache/pro_1000'

loader = ProMatchesDataloader()
loader.load(path=data_path)

data = loader.data

None


## Готовим датасет

0 - Radiant \
1 - Dire

In [154]:
pb = [elem.picks_bans for elem in data]

radiants, dires, targets = [], [], []

for i, elem in enumerate(data):
    pb = elem.picks_bans
    target = elem.pro_match_data.radiant_win

    if pb is None or target is None:
        continue
    
    pb = sorted(filter(lambda x: x.is_pick, pb), key=lambda x: x.team)
    assert len(pb) == 10

    radiants.append([e.hero_id for e in pb[:5]])
    dires.append([e.hero_id for e in pb[5:]])
    targets.append(int(target))

len(targets)

989

In [159]:
df = pd.DataFrame()
df['target'] = targets
df['radiant_hero_ids'] = radiants
df['dire_hero_ids'] = dires

display(df.sample(10))
df.groupby('target')['target'].count()

,target,radiant_hero_ids,dire_hero_ids
982,0,"[19, 110, 102, 112, 90]","[84, 27, 49, 70, 135]"
381,0,"[18, 86, 97, 26, 35]","[14, 56, 80, 11, 48]"
376,1,"[5, 43, 8, 17, 114]","[83, 86, 96, 10, 39]"
577,1,"[123, 66, 10, 78, 7]","[96, 41, 128, 68, 82]"
260,0,"[105, 123, 54, 49, 126]","[69, 72, 65, 15, 120]"
555,1,"[137, 79, 106, 87, 10]","[100, 21, 98, 121, 70]"
274,1,"[87, 9, 2, 25, 48]","[27, 21, 51, 14, 8]"
216,0,"[5, 26, 96, 13, 10]","[84, 86, 129, 7, 44]"
690,1,"[40, 97, 111, 46, 18]","[105, 86, 104, 89, 49]"
748,1,"[97, 86, 79, 96, 93]","[20, 71, 46, 28, 106]"


target
0    481
1    508
Name: target, dtype: int64

# В DIRE проебался один айдишник, 

### надо намайнить больше данных или пофиксить алгоритмически

In [160]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(sparse_output=True)

df1 = df.join(
        pd.DataFrame.sparse.from_spmatrix(
            mlb.fit_transform(df.pop('radiant_hero_ids')),
            index=df.index,
            columns=mlb.classes_
        )
    )
display(df1)



df2 = df1.join(
        pd.DataFrame.sparse.from_spmatrix(
            mlb.fit_transform(df1.pop('dire_hero_ids')),
            index=df1.index,
            columns=mlb.classes_
        ),
        lsuffix='_radiant', 
        rsuffix='_dire'
    )
display(df2)

,target,dire_hero_ids,1,2,3,4,5,6,7,8,...,120,121,123,126,128,129,135,136,137,138
0,0,"[121, 126, 21, 105, 135]",0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,1,"[53, 128, 70, 106, 129]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,"[49, 9, 105, 93, 50]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,"[123, 79, 96, 41, 61]",0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,"[121, 49, 19, 54, 21]",0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984,0,"[31, 19, 26, 11, 112]",0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
985,0,"[19, 66, 49, 12, 120]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
986,0,"[45, 31, 54, 28, 19]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
987,0,"[5, 112, 99, 42, 52]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,target,1_radiant,2_radiant,3_radiant,4_radiant,5_radiant,6_radiant,7_radiant,8_radiant,9_radiant,...,120_dire,121_dire,123_dire,126_dire,128_dire,129_dire,135_dire,136_dire,137_dire,138_dire
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,1,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
985,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
986,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
987,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Дропаем лишние айдишники из колонок (тк всего героев 124 а пронумерованы они до 138)